# Sandbox
This notebook covers the main steps in deploying a model.  Experimentation with features is encouraged.  Can you increase the NDCG reported during model training?

## Setup Client

In [ ]:
from ltr.client import ElasticClient
client = ElasticClient()

## Step 1 - Create a Feature Set

In [ ]:
'''
  TASK:
  Experiment with this featureset, the judgments are all title searches so what features could be useful?
  
  Ideas:
    - Search other fields
    - Phrase matches
    - Fuzzy matches
'''

client.reset_ltr(index='tmdb')

config = {
   "featureset": {
        "features": [
            {
                "name": "title_match",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    }
                }
            }
        ]
    }
}

client.create_featureset(index='tmdb', name='sandbox', ftr_config=config)

## Step 2 - Log Features for Training

In [ ]:
from ltr.judgments import judgments_open
from ltr.log import log_query
from itertools import groupby

training_set=[]
with judgments_open('data/title_judgments.txt') as judg_list:
    for qid, query_judgments in groupby(judg_list, key=lambda j: j.qid):
        query_training_set, discarded = log_query(client, index='tmdb', 
                                                  judgments=query_judgments, feature_set='sandbox')
        training_set.extend(query_training_set)

## Step 3 - Train a Model

In [ ]:
'''
  TASK:
  Experiment with the leafs and trees variables, how do they affect NGCG?
  Does a high leaf value increase your NDCG?  What could be the potential downfalls?
'''
from ltr.ranklib import train
trainLog  = train(client,
                  index='tmdb',
                  training_set=training_set,
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  featureSet='sandbox',
                  modelName='sandbox')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

## Search

In [ ]:
from ltr import search
search(client, "rambo", modelName='sandbox')